# IBM Staff Retreat To Toronto

## IBM Applied Data Science Capstone Week 4 Peer Graded Assignment

## Report

## Introduction

A staff retreat is a perfect opportunity to take a step back from the day-to-day work and spend some time thinking about the bigger picture. It's a time to rejuvenate the team, get everyone excited around a united mission, set goals for the upcoming year, and build bonds between team members. 

IBM is organising a staff retreat for its employeed this weekend.To make sure that this event goes smoothly, IBM has to come up with a list of prospective hotels, restaurants, attractions, shopping centers and parks to book.

Taking this into consideration, I will be making a list of all the possible locations to book and superimpose the on a map of Toronto to make it easier to visualise, thereby making it easier to pick and choose which locations to book.

## Data

All the data in this project comes from the Foursquare API. Location data of all places will be taken from the Foursquare database and displayed on a map using the Folium library.

## Methodology

1. Importing Libraries
2. Defining Foursquare Credentials And Version
3. Defining Toronto's Location
4. Making A List Of Hotels
5. Making A List Of Restaurants
6. Making A List Of Shopping Centers
7. Making A List Of Parks
8. Making A List Of Theaters
9. Visualising All The Data On A Map

#### Importing Libraries

Libraries pertinant to this project will be imported, for example: Folium, Numpy, Pandas, Geopy etc. These will be used to process the data acquired from the Fooursquare API and display it.

#### Define Foursquare Credentials And Version And Making Lists/Dataframes

After the Foursuqare credentials and version are defined, they will be used to make calls to the Foursquare API in order to retrieve data about all the different places IBM has to make lists for.

#### Visualising All The Data On A Map

The folium library and dataframes of all the locations will be used in order to create a map of Toronto and superimpose the location of all the places on it in the form of multi-coloured circles (each representing a place belonging to a different list).

## Results

The reterival of all the data and its plotting allowed the event organisers to get a good idea of all prospective locations that can become a part of the staff retreat.

## Discussion

The resulting map contains all the locations of the places to visit and allows us to easily calculate the distance between them. This can help in planning the retreat. We can draw a circle of radius 1 Km around each of the hotels and book all the unqiue places that fall with the circle for the retreat.

After looking at the map, I have decided that **Marriott Downtown at CF Toronto Eaton Centre, 525 Bay Street** is the best hotel to book. It has all the places like parks and theaters nearby, unlike the other hotels.

## Conclusion

Using the Foursquare API and python libraries, we were able to obtain location data on prospective places to book for the retreat. This allowed me to visualise it, which in turn made it easier to plan out the retreat.

## Code

### 1. Importing Libraries

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

### 2. Defining Foursquare Credentials And Version

In [37]:
CLIENT_ID = 'GORPKUDCXIUAPR3ZEUYBOHELQPDHICFYNML0XWLGFOAFDCZ1'
CLIENT_SECRET = 'AIGORMYWPA5IFMAH5LQBVOACLVACSC3FIZYKD5JKF0TR51F0' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GORPKUDCXIUAPR3ZEUYBOHELQPDHICFYNML0XWLGFOAFDCZ1
CLIENT_SECRET:AIGORMYWPA5IFMAH5LQBVOACLVACSC3FIZYKD5JKF0TR51F0


### 3. Defining Toronto's Locations

In [38]:
city = 'Toronto'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(city)
longitude = location.longitude
latitude = location.latitude
print("The Latitude And Longitude of Toronto Are: {} , {}".format(latitude, longitude))

The Latitude And Longitude of Toronto Are: 43.6534817 , -79.3839347


### 4. Making A List Of Hotels

In [176]:
search_query = 'Hotel'
radius = 500
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

In [177]:
venues = results['response']['venues']

dataframe = json_normalize(venues)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [178]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]



# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [179]:
dataframe_filtered2 = dataframe_filtered.drop(['cc','state', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress','labeledLatLngs','neighborhood', 'id'], axis=1)

In [186]:
dataframe_filtered3 = dataframe_filtered2.dropna()
array1 = ['Hotel']
df_hotels = dataframe_filtered3.loc[dataframe_filtered3['categories'].isin(array1)]
df_hotels.drop_duplicates(subset = 'name', keep = 'first', inplace = True)
df_hotels.index.name = None
df_hotels

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,name,categories,address,lat,lng,postalCode
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4
3,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1
11,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3
18,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3
24,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3
26,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2
29,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2


### 5. Making A List Of Restaurants

In [213]:
search_query = 'Restaurant'
radius = 500
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

In [214]:
venues = results['response']['venues']

dataframe = json_normalize(venues)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [215]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]



# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [217]:
dataframe_filtered2 = dataframe_filtered.drop(['cc','state', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress','labeledLatLngs','neighborhood', 'id'], axis=1)
df_rest = dataframe_filtered2.dropna(axis = 0, how='any', thresh=None, subset=None, inplace=False)
df_rest

,name,categories,address,lat,lng,postalCode
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3
1,Restaurant | Adelaide,Restaurant,325 Bay St (31st floor),43.649563,-79.380590,M5H
2,Osgoode Hall Restaurant,New American Restaurant,130 Queen St W,43.651979,-79.385049,M5H 2N6
3,Tundra Restaurant,Restaurant,145 Richmond Street West,43.650069,-79.385409,M5H 2L2
4,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,43.654925,-79.387089,M5G 1C7
6,Cali Restaurant,Vietnamese Restaurant,179 Dundas St. W.,43.655068,-79.386375,M5G
7,Maezo Restaurant,Indian Restaurant,67 Richmond St West,43.651395,-79.381888,M5H 1W2
9,Spring Rolls | Japanese Restaurant in Toronto,Theme Restaurant,"40 Dundas St W Box 36,",43.656105,-79.383495,M5G 2C2
11,Maezo Restaurant & Bar,Indian Restaurant,67 Richmond Street West,43.651091,-79.381843,M5H 1Z5
12,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,43.652614,-79.380231,M5H 3X4


### 6. Making A List Of Shopping Locations

In [218]:
search_query = 'Mall'
radius = 500
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

venues = results['response']['venues']

dataframe = json_normalize(venues)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


In [219]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]



# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [220]:
dataframe_filtered2 = dataframe_filtered.drop(['cc','state', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress','labeledLatLngs','neighborhood', 'id'], axis=1)
array2 = ['Shopping Mall']
df_shop = dataframe_filtered2.loc[dataframe_filtered2['categories'].isin(array2)]
df_shop

,name,categories,lat,lng,address,postalCode
1,CF Toronto Eaton Centre,Shopping Mall,43.65454,-79.380677,220 Yonge St,M5B 2H1


### 7. Making A List Of Parks

In [221]:
search_query = 'Park'
radius = 500
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

venues = results['response']['venues']

dataframe = json_normalize(venues)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


In [222]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]



# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [223]:
dataframe_filtered2 = dataframe_filtered.drop(['cc','state', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress','labeledLatLngs','neighborhood', 'id'], axis=1)
array3 = ['Plaza', 'Park']
dataframe_filtered3 = dataframe_filtered2.loc[dataframe_filtered2['categories'].isin(array3)]
df_park = dataframe_filtered3.dropna()
df_park

,name,categories,lat,lng,address,postalCode
3,Trinity Square Park,Plaza,43.654379,-79.381849,19 Trinity Square,M5G 1B1
4,Osgoode Hall Park,Park,43.651105,-79.386269,130 Queen St W,M5H 2N5
17,Butterfield Park,Park,43.652697,-79.391091,100 McCaul St,M5T 1W1


### 8. Making A List Of Theaters

In [224]:
search_query = 'Theater'
radius = 500
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

venues = results['response']['venues']

dataframe = json_normalize(venues)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


In [225]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]



# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [229]:
dataframe_filtered2 = dataframe_filtered2 = dataframe_filtered.drop(['cc','state', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress','labeledLatLngs','id'], axis=1)
array4 = ['Multiplex', 'Theater', 'Movie']
dataframe_filtered3 = dataframe_filtered2.loc[dataframe_filtered2['categories'].isin(array4)]
df_theater = dataframe_filtered3.dropna()
df_theater

,name,categories,address,lat,lng,postalCode
0,Theater 9,Multiplex,10 Dundas St. E.,43.656804,-79.380405,M5B 1R4
2,Elgin And Winter Garden Theatres,Theater,189 Yonge St,43.653394,-79.378507,M5B 2H1
3,Ed Mirvish Theatre,Theater,244 Victoria St.,43.655102,-79.379768,M5B 1V8


### 9. Visualising All The Data On A Map

In [257]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, label, name, address in zip(df_hotels.lat, df_hotels.lng, df_hotels.categories, df_hotels.name, df_hotels.address):
    label = '{}, {}, {}'.format(name, address, label)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=7,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=1
    ).add_to(venues_map)

for lat, lng, label, name, address in zip(df_rest.lat, df_rest.lng, df_rest.categories, df_rest.name, df_rest.address):
    label = '{}, {}, {}'.format(name, address, label)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=4,
        color='black',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

for lat, lng, label, name, address in zip(df_shop.lat, df_shop.lng, df_shop.categories, df_shop.name, df_shop.address):
    label = '{}, {}, {}'.format(name, address, label)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)

for lat, lng, label, name, address in zip(df_park.lat, df_park.lng, df_park.categories, df_park.name, df_park.address):
    label = '{}, {}, {}'.format(name, address, label)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=1
    ).add_to(venues_map)

for lat, lng, label, name, address in zip(df_theater.lat, df_theater.lng, df_theater.categories, df_theater.name, df_theater.address):
    label = '{}, {}, {}'.format(name, address, label)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='white',
        fill_opacity=1
    ).add_to(venues_map)
venues_map

The map is here:
https://github.com/nachiket-san/Coursera_Capstone/blob/master/Map.png
clicking on the circle marker